In [1]:
% matplotlib inline

import numpy as np
import numpy.ma as ma
import pandas as pd
from math import *
import matplotlib.pyplot as plt
import scipy.io as sio
from mpl_toolkits.basemap import Basemap, cm

import MITgcmutils as mit
from MITgcmutils import cs

from matplotlib import rcParams, rc
rcParams.update({'figure.autolayout': True})
rcParams.update({'font.family': 'serif'})
rcParams.update({'font.serif': 'Times New Roman'})



/Users/sclayton/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
## import the mean data

CRindir = '/Volumes/My Passport/coarse_run/'
HRindir = '/Volumes/My Passport/high_res/'

#import grid params for CR grid

in_file = '%sgrid/HFacC.data' % CRindir
HFCR = np.fromfile(in_file, dtype = '>f')
HFCR = HFCR.reshape(23, 160, 360)

in_file = '%sgrid/XC.data' % CRindir
XC = np.fromfile(in_file, dtype = '>f')
XC = XC.reshape(160,360)

in_file = '%sgrid/YC.data' % CRindir
YC = np.fromfile(in_file, dtype = '>f')
YC = YC.reshape(160,360)

in_file = '/Users/sclayton/Documents/darwin_model/eg_grid/RA.data'
RA = np.fromfile(in_file, dtype = '>f')
RA = RA.reshape(160,360)

## pick a nutrient 0 = phosphate, 1 = nitrate, 2 = iron, 3 = silica
v=2

# import CR data

in_file = '%supflux/CR_WN.1999.data' % CRindir
CRWN = np.fromfile(in_file, dtype='>f').reshape(4,160, 360)
CRmask = ma.masked_where(HFCR[6,:,:] == 0, CRWN[v,:,:], copy=False)

#import grid params for HR grid
HFHR = mit.rdmds('/Users/sclayton/Documents/darwin_model/e2_grid/hFacC')
XG = mit.rdmds('/Users/sclayton/Documents/darwin_model/e2_grid/XG')
YG = mit.rdmds('/Users/sclayton/Documents/darwin_model/e2_grid/YG')
RAC = mit.rdmds('/Users/sclayton/Documents/darwin_model/e2_grid/RAC')

# import HR data
in_file = '%supflux/WN.1999.data' % HRindir
HRWN = np.fromfile(in_file, dtype='>f').reshape(4, 510, 3060)
HRmask = ma.masked_where(HFHR[10,:,:] == 0, HRWN[v,:,:], copy=False)

for n in range(4):
    CRWN[n, :, :] = ma.masked_where(HFCR[6,:,:] == 0, CRWN[n,:,:], copy=True)
    HRWN[n, :, :] = ma.masked_where(HFHR[10,:,:] == 0, HRWN[n,:,:], copy=True)

In [7]:
## calculate the eddy upflux

HRWNanom = np.zeros((4, 12, 510, 3060))
CRWNanom = np.zeros((4, 12, 160, 360))

d = 1
nd = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
ed = np.cumsum(nd)
sd = np.ones(12)
sd[1:12] = ed[0:11]
sd = sd.astype(int)

for m in range(12):
    for d in range(sd[m], ed[m]+1):
    
        infile = '%supflux/WN.1999.%03d.data' % (HRindir, d)
        tmph = np.fromfile(infile, dtype = '>f').reshape(4,510, 3060)

        infile = '%supflux/CR_WN.1999.%03d.data' % (CRindir, d)
        tmpc = np.fromfile(infile, dtype = '>f').reshape(4,160,360)

        for n in range(4):
            HRWNanom[n,m,:,:] =+ ma.masked_where(HFHR[0,:,:] == 0, tmph[n,:,:], copy=True) - HRWN[n,:,:]
            CRWNanom[n,m,:,:] =+ ma.masked_where(HFCR[6,:,:] == 0, tmpc[n,:,:], copy=True) - CRWN[n,:,:]
            
    HRWNanom[:,m,:,:]=HRWNanom[:,m,:,:]/nd[m]
    CRWNanom[:,m,:,:]=CRWNanom[:,m,:,:]/nd[m]
    

In [ ]:
# work out the latitudinally averaged vertical nutrient supply

n = 1
m = 6

# EG
CRHFmask = CRWNanom[n,m,:,:] *HFCR[6,:,:]*RA
binsize = 5
ybin = np.arange(-60, 60+binsize, binsize)

binnedcr = np.empty([len(ybin)])
numobs = np.empty([len(ybin)])
areacr = np.empty([len(ybin)])

for i in range(len(ybin)):
    lat = ybin[i]

    nn = len((YC>=lat-binsize/2) & (YC<lat +binsize/2))
    if nn > 0:
        #print YC[[(YC>(lat-binsize/2)) & (YC<(lat +binsize/2))]]
        areacr[i] = np.nansum(RA[(YC>=lat-binsize/2) & (YC<lat +binsize/2)])
        binnedcr[i] = np.nansum(CRHFmask[(YC>=lat-binsize/2) & (YC<lat +binsize/2)])/areacr[i]
        numobs[i] = nn
    
    else:
        binnedcr[i] = np.nan
        numobs[i] = np.nan

# E2
HRHFmask = HRWNanom[n,m,:,:] *HFHR[10,:,:]*RAC

binnedhr = np.empty([len(ybin)])
numobs = np.empty([len(ybin)])
areahr = np.empty([len(ybin)])

for i in range(len(ybin)):
    lat = ybin[i]

    nn = len((YG>=lat-binsize/2) & (YG<lat +binsize/2))
    if nn > 0:
        #print YC[[(YC>(lat-binsize/2)) & (YC<(lat +binsize/2))]]
        areahr[i] = np.nansum(RAC[(YG>=lat-binsize/2) & (YG<lat +binsize/2)])
        binnedhr[i] = np.nansum(HRHFmask[(YG>=lat-binsize/2) & (YG<lat +binsize/2)])/areahr[i]
        numobs[i] = nn
    
    else:
        binnedhr[i] = np.nan
        numobs[i] = np.nan

s_to_y = 365 

fig3 = plt.figure(3, figsize=(9,9), dpi = 600)

plt.plot(binnedcr*s_to_y, ybin,'.-r', label = 'CR', lw =2)
plt.plot(binnedhr*s_to_y, ybin,'.-b',label= 'HR',lw = 2)
plt.ylabel('Latitude', fontsize=20)
plt.legend(fontsize = 20)
plt.tick_params(axis='both', which='major', labelsize=20)
plt.xlabel(r'Annual Average vertical flux of NO3 at 100m', fontsize=20)
plt.axvline(0, color='k')
plt.axhline(35, color='k')
plt.axhline(5, color='k')
plt.axhline(-5, color='k')
plt.axhline(-35, color='k')
plt.axis([-500, 1000, -60, 60])
plt.show()


In [ ]:
# latmin = [35, 5, -5, -35, -60]
# latmax = [60, 35, 5, -5, -35]

# region = ['N SPolar', 'N STropic', 'Eq', 'S STropic', 'S SPolar']


# for l in range(len(latmax)):
#     HRint = HRWNanom[v,:,:]*HFHR[0,:,:]*RAC
#     subset = HRint[(YG>latmin[l]) & (YG<latmax[l])]
    
#     print region[l]
#     print 'ECCO2', np.nansum(subset)/(np.sum(RAC[(YG>latmin[l]) & (YG<latmax[l])]))

#     CRint = CRWNanom[v,:,:]*HFCR[6,:,:]*RA
#     subset = CRint[(YC>latmin[l]) & (YC<latmax[l])]
#     print 'ECCO-Godae', np.nansum(subset)/(np.sum(RA[(YC>latmin[l]) & (YC<latmax[l])]))
#     print '\n'